# Config

In [ ]:
WB_CONFIG = {
    "epochs": 10,
    "batch_size": 16,
    "encoder_weights": "imagenet",
    "split_ratio": (0.8, 0.1, 0.1),
    "image_size": (512, 512),
    "learning_rate": 0.0001,
}
INTERNAL_CONFIG = WB_CONFIG.copy()
BUFFER_SIZE = 512

# Dataset initialization

## Helpers

In [ ]:
import os
import tensorflow as tf

def make_config(ds_size: int, split_ratio: tuple, batch_size: float) -> dict:
    assert sum(split_ratio) == 1
    train_size = int(ds_size * split_ratio[0])
    val_size = round(ds_size * split_ratio[1])
    test_size = round(ds_size * split_ratio[2])
    return {
        "train_size": train_size,
        "val_size": val_size,
        "test_size": test_size,
        "steps_per_epoch": train_size // batch_size,
        "val_steps": val_size // batch_size
    }

def load_image_paths(data_path: str):
    # Load tumor image paths with their labels
    tumors_data_path = data_path + "/tumors/images/"
    image_paths = [tumors_data_path + im_path for im_path in os.listdir(tumors_data_path)]
    labels = [0 if "empty" in im_path else 1 for im_path in image_paths]
   
    # Load nerve image paths with 0 labels (do not contain tumor)
    nerves_data_path = data_path + "/nerves/images/"
    nerve_image_paths = [nerves_data_path + im_path for im_path in os.listdir(nerves_data_path)]
    image_paths = nerve_image_paths + image_paths
    labels = [0 for _ in nerve_image_paths] + labels
    
    # Load PNI image paths with 1 labels (contain tumor)
    pni_data_path = data_path + "/pni/images/"
    pni_image_paths = [pni_data_path + im_path for im_path in os.listdir(pni_data_path)]
    image_paths = pni_image_paths  + image_paths
    labels = [1 for _ in pni_image_paths] + labels

    # Convert arrays to tensors
    image_paths = tf.convert_to_tensor(image_paths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)
    
    return image_paths, labels


# Returns the function that loads a single image and its corresponding label from the provided paths
def load_images(image_size: tuple=(512,512)):
    def load_images_func(image_path, label):
        image = tf.io.read_file(image_path)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, image_size)
        return image, label

    return load_images_func


# Splits the dataset into 2 parts based on the train_ratio and returns these 2 parts
def split(dataset, train_size: int):
    dataset = dataset.shuffle(buffer_size=BUFFER_SIZE)
    return dataset.take(train_size), dataset.skip(train_size)

## Load dataset

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

DATA_PATH = "/kaggle/input/masters-thesis-extended/data/pancreas"

image_paths, labels = load_image_paths(DATA_PATH)

dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset = dataset.map(load_images(INTERNAL_CONFIG["image_size"]))
ds_size = dataset.cardinality().numpy()

INTERNAL_CONFIG.update(make_config(ds_size, INTERNAL_CONFIG["split_ratio"], INTERNAL_CONFIG["batch_size"]))

train_ds, val_ds = split(dataset, INTERNAL_CONFIG["train_size"])
val_ds, test_ds = split(val_ds, INTERNAL_CONFIG["val_size"])

train_batches = (train_ds.shuffle(BUFFER_SIZE)
                         .batch(INTERNAL_CONFIG["batch_size"])
                         .repeat()
                         .prefetch(buffer_size=tf.data.AUTOTUNE))
val_batches = val_ds.batch(INTERNAL_CONFIG["batch_size"])
test_batches = test_ds.batch(INTERNAL_CONFIG["batch_size"])

# Display cardinality
labels, counts = np.unique(np.fromiter(dataset.map(lambda x, y: y), np.int32), return_counts=True)
labels = ["non-tumor" if l == 0 else "tumor" for l in labels]
plt.ylabel('Counts')
plt.xlabel('Labels')
ax = sns.barplot(x=labels, y=counts)
ax.bar_label(ax.containers[0])

print(f"TRAIN: {train_ds.cardinality()}\nVAL: {val_ds.cardinality()}\nTEST: {test_ds.cardinality()}")

# Tumor classification model

## Prepare model

In [ ]:
# Prepare base model for feature extraction 
model = tf.keras.applications.ResNet50V2(input_shape=(*INTERNAL_CONFIG["image_size"], 3), include_top=False, weights=INTERNAL_CONFIG["encoder_weights"])
model.trainable = INTERNAL_CONFIG["encoder_weights"] is None

# Add fully connected layers for classification
x = model.output
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.models.Model(inputs=model.input, outputs=predictions)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(INTERNAL_CONFIG["learning_rate"]), metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

## Display architecture

In [ ]:
!pip install -q git+https://github.com/paulgavrikov/visualkeras
import visualkeras
from PIL import ImageFont

color_map = {
    tf.keras.layers.InputLayer: {"fill": "gray"},
    tf.keras.layers.ZeroPadding2D: {"fill": "#06d6a0"},
    tf.keras.layers.Conv2D: {"fill": "#ffd166"},
    tf.keras.layers.MaxPooling2D: {"fill": "#de7b1f"},
    tf.keras.layers.BatchNormalization: {"fill": "#118ab2"},
    tf.keras.layers.Add: {"fill": "#ef476f"},
    tf.keras.layers.Dense: {"fill": "#40a62b"},
    tf.keras.layers.GlobalAveragePooling2D: {"fill": "#073b4c"},
}

font=ImageFont.truetype("DejaVuSansMono.ttf", 120)
visualkeras.layered_view(model, legend=True, font=font, color_map=color_map, type_ignore=[tf.keras.layers.Activation], scale_xy=12, max_xy=4000, min_xy=40)

# Training

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

wandb.login(key=user_secrets.get_secret("wandb-api-key"))
wandb.init(project="tumor-detection", entity="xjackor", config=WB_CONFIG)

model.fit(train_batches,
          epochs=INTERNAL_CONFIG["epochs"],
          steps_per_epoch=INTERNAL_CONFIG["steps_per_epoch"],
          validation_steps=INTERNAL_CONFIG["val_steps"],
          validation_data=val_batches,
          callbacks=[wandb.keras.WandbCallback()])

wandb.run.finish()

# Evaluation

In [ ]:
model.evaluate(test_batches)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

labels, labels_pred = [], []
for image, label in test_batches.unbatch():
    labels_pred.append(model.predict(tf.expand_dims(image, axis=0), verbose=0)[0][0])
    labels.append(label.numpy())


cm = confusion_matrix(labels, np.round(labels_pred), normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
